### IMPORTS

In [1]:
import json
from web3 import Web3

### LOAD SETTINGS & LATEST ABI FILE

In [2]:
with open('settings.json') as json_file:
    settings = json.load(json_file)

In [3]:
with open('latest.json') as json_file:
    latest = json.load(json_file)

### IPFS & BLOCKCHAIN GATEWAY CONNECTIONS

In [4]:
class gateway:
    def __init__(self, foo):
        self.host = foo['host']
        self.port = foo['port']

In [5]:
blockchain = gateway(settings['gateways']['blockchain'])

In [6]:
ipfs = gateway(settings['gateways']['ipfs'])

### BLOCKCHAIN PUBLIC/PRIVATE KEYS

In [7]:
class keypair:
    def __init__(self, foo):
        self.public = foo['public']
        self.private = foo['private']

In [8]:
keys = keypair(settings['keys'])

### CONNECT TO CHAIN VIA WEBSOCKET

In [9]:
web3 = Web3(Web3.WebsocketProvider('ws://' + blockchain.host + ':' + blockchain.port))

In [10]:
web3.isConnected()

True

### CONSTRUCT USABLE CONTRACTS

In [177]:
class manager:
    
    # ON LOAD..
    def __init__(self, block):
        
        # STITCH CONTRACT TOGETHER
        self.contract = web3.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, func):
        return self.contract.functions[func]().call()
    
    # WRITE TO CONTRACT
    def write(self, details):
        try:
            
            # CREATE TRANSACTION
            tx = {
                'from': keys.public,
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                ),
                'gas': 2000000,
                'gasPrice': web3.toWei('1', 'gwei'),
                'nonce': web3.eth.getTransactionCount(keys.public)
            }

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = web3.eth.account.sign_transaction(tx,
                private_key = keys.private
            )

            # SEND THE TRANSACTION
            web3.eth.sendRawTransaction(signed.rawTransaction)

            # SUCCESS
            return 'success'
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error

### CONTRACT REFERENCES

In [178]:
user_manager = manager(latest['usermanager'])

In [179]:
device_manager = manager(latest['devicemanager'])

In [180]:
task_manager = manager(latest['taskmanager'])

In [181]:
token_manager = manager(latest['tokenmanager'])

### TESTING

In [182]:
token_manager.read('price')

5

In [183]:
user_manager.read('fetch_everyone')

[]

In [185]:
token_manager.write({
    'func': 'init',
    'params': [5, task_manager.address]
})

ValueError({'message': 'VM Exception while processing transaction: revert contract has already been initialized',
            'code': -32000,
            'data': {'0x1e214e5de4aa1b4a54d93db5c70b7a7c2a4ba1a12f8164f91234670e8d79b0ea': {'error': 'revert',
              'program_counter': 832,
              'return': '0x08c379a000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000025636f6e74726163742068617320616c7265616479206265656e20696e697469616c697a6564000000000000000000000000000000000000000000000000000000',
              'reason': 'contract has already been initialized'},
             'stack': 'o: VM Exception while processing transaction: revert contract has already been initialized\n    at Function.o.fromResults (C:\\Users\\35840\\AppData\\Roaming\\npm\\node_modules\\ganache-cli\\build\\ganache-core.node.cli.js:10:89727)\n    at C:\\Users\\35840\\AppData\\Roaming\\npm\\node_modules\\ganache-cli\\build\\ganache-co